In [ ]:
import csv
import mysql.connector
import pandas as pd

# MySQL database connection parameters
db_config = {
    "host": "", # localhost
    "user": "",
    "password": "",
    "database": "",
}

# Extract data from file
def extract_data(input_file):
    data = []
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            data.append((row[0], row[1]))
    return transform_data(data)

# Cleans data
def transform_data(data):

    df = pd.DataFrame(data)
    df.columns = ['Movie','Year']
    df = df[df['Year'].apply(lambda x: x.isnumeric())]
    df = df.values.tolist()
    return df


# Loads data into a MySQL database
def load_data_to_mysql(data, connection_params):
    try:
        # Establish a connection to the MySQL database
        connection = mysql.connector.connect(**connection_params)

        # Create a cursor object to interact with the database
        cursor = connection.cursor()

        # Create a table if it doesn't exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS movie (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            year INT
        )
        """
        cursor.execute(create_table_query)

        # Insert data into the table
        insert_query = "INSERT INTO movies (title, year) VALUES (%s, %s)"
        cursor.executemany(insert_query, data)

        # Commit changes and close the cursor and connection
        connection.commit()
        cursor.close()
        connection.close()

        print("Data loaded into MySQL successfully!")

    except mysql.connector.Error as error:
        print("Error:", error)

if __name__ == "__main__":
    # File with data
    filepath = r"C:\Users\F\Downloads\show_list.csv"


    # Extract and Transform. 
    extracted_data = extract_data(filepath)

    # Load the extracted data into the MySQL database
    load_data_to_mysql(extracted_data, db_config)
